In [18]:
import pandas as pd
import os
import re
import numpy as np
from params.paths import ROOT_DIR
from api_requests.meeting_convo_collector import MeetingConvoCollector
from file_handling.file_read_writer import read_json, write_json, create_dir, write_file

In [19]:
#params
DATA_DIR = os.path.join(ROOT_DIR, 'data')
RESOURCE_DIR = os.path.join(ROOT_DIR, 'resource')

In [20]:
#loading the labels corresponding to the label ids
label2id = read_json(os.path.join(RESOURCE_DIR, "labels.json"))
id2label = {v:k for k, v in label2id.items()}
label2id = {k: int(v) for k, v in label2id.items()}
for k, v in label2id.items():
    print(f"{k}=>{v}")
print("___________________________________________________________")
for k, v in id2label.items():
	print(f"{k}=>{v}")

事実文=>0
質問文=>1
説明文=>2
意見文=>3
その他=>4
___________________________________________________________
0=>事実文
1=>質問文
2=>説明文
3=>意見文
4=>その他


# Preparing dataset

In [21]:
from datasets import load_dataset
PATH_TO_DATA_FILE = os.path.join(DATA_DIR, "labelled_data_77.csv")
dataset = load_dataset('csv', data_files=PATH_TO_DATA_FILE, split="train").train_test_split(test_size=0.2)
dataset = dataset.filter(lambda example: bool(example["speech"])).filter(lambda example: len(example['speech'])<1000)
dataset.rename_column("label", "label_name")
dataset = dataset.rename_columns({"label":"label_name", "label_id":"label"})
print(dataset)
print(dataset["train"][199])

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-33ac082739253181/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/csv/default-33ac082739253181/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-66fd28b0850fe643.arrow and /root/.cache/huggingface/datasets/csv/default-33ac082739253181/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-f2883ac8d1b12ffb.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-33ac082739253181/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-d4ec77b604a93a06.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-33ac082739253181/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-ee07986f5469986f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datase

DatasetDict({
    train: Dataset({
        features: ['speech', 'label_name', 'label', 'record_position'],
        num_rows: 1138
    })
    test: Dataset({
        features: ['speech', 'label_name', 'label', 'record_position'],
        num_rows: 284
    })
})
{'speech': '八日の参議院法務委員会では、委員長職権による政府案の強行採決が行われ、到底通常の委員会運営ではなかったことに厳しく抗議をいたします', 'label_name': '意見文', 'label': 3, 'record_position': 24}


In [22]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v3")

def tokenize_function(examples):
    result = tokenizer(examples["speech"], truncation=True, padding=True)
    return result

tokenized_datasets = dataset.map(tokenize_function, batched=True)

ModuleNotFoundError: The ipadic dictionary is not installed. See https://github.com/polm/ipadic-py for installation.

# Training

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-v3", num_labels=5, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-large-japanese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    label_names=['label_id']
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

/root/projects/mi_projects/jp_speech_categorization/speech_cat_venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 8.00 GiB total capacity; 7.13 GiB already allocated; 0 bytes free; 7.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF